In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [66]:
binned = pd.read_csv('/Users/flatironschool/Absenteeism_Project/data/processed/combo_cleaned.csv')

In [67]:
binned.head()

,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,LEA_STATE_NAME,...,sports_part,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate
0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,ALABAMA,...,220.0,0.169884,78.0,0.060232,0.0,0.000000,2811.937359,0.0,0.003475,0.378788
1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,GE95,AL,ALABAMA,...,175.0,0.324675,10.0,0.018553,2.0,0.003711,4825.189777,0.0,0.002783,0.100000
2,2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,AL,ALABAMA,...,229.0,0.385522,18.0,0.030303,5.0,0.008418,5317.932795,0.0,0.001684,0.105263
3,3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,AL,ALABAMA,...,241.0,0.525054,10.0,0.021786,0.0,0.000000,5909.375686,0.0,0.002179,0.068966
4,4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,AL,ALABAMA,...,64.0,0.176309,8.0,0.022039,0.0,0.000000,3962.305785,0.0,0.002755,0.101010


## Imputed mean for missing values 

In [117]:
#impute mean for numerical vars
binned_feat = binned[['total_enrollment', 'ap_ib_de_rate', 'sat_act_rate', 'pass_algebra_rate',
      'geometry_rate', 'algebra2_rate', 'calc_rate', 'chronic_absent_rate', 'activities_funds_rate',
      'sports_rate', 'suspensed_day_rate', 'harassed_rate',
      'non_cert_rate','counselor_rate','absent_teacher_rate']]

In [118]:
imputed_numeric_df = binned_feat
for i in range(0, len(binned_feat.columns)):
    print('im here:', binned_feat.columns[i])
    imputed_numeric_df.iloc[:,i].replace(np.NaN, imputed_numeric_df.iloc[:,i].mean(), inplace=True)

im here: total_enrollment
im here: ap_ib_de_rate
im here: sat_act_rate
im here: pass_algebra_rate
im here: geometry_rate
im here: algebra2_rate
im here: calc_rate
im here: chronic_absent_rate
im here: activities_funds_rate
im here: sports_rate
im here: suspensed_day_rate
im here: harassed_rate
im here: non_cert_rate
im here: counselor_rate
im here: absent_teacher_rate


In [119]:
#No missing values
imputed_numeric_df.isna().sum()

total_enrollment         0
ap_ib_de_rate            0
sat_act_rate             0
pass_algebra_rate        0
geometry_rate            0
algebra2_rate            0
calc_rate                0
chronic_absent_rate      0
activities_funds_rate    0
sports_rate              0
suspensed_day_rate       0
harassed_rate            0
non_cert_rate            0
counselor_rate           0
absent_teacher_rate      0
dtype: int64

## Binning Graduation Rates into 10 Rates



In [81]:
#separate out schools that have 31 or more students in the graduation class
bigger = binned[binned['ALL_COHORT_1516'] >= 31]

In [82]:
#remove "GE" and "LE" from ranges
bigger.replace({'ALL_RATE_1516': 'GE95'}, '95', inplace=True)
bigger.replace({'ALL_RATE_1516': 'GE90'}, '90', inplace=True)
bigger.replace({'ALL_RATE_1516': 'GE99'}, '99', inplace=True)
bigger.replace({'ALL_RATE_1516': 'LE10'}, '10', inplace=True)
bigger.replace({'ALL_RATE_1516': 'LE1'}, '1', inplace=True)
bigger.replace({'ALL_RATE_1516': 'LE5'}, '5', inplace=True)


/anaconda3/envs/absenteeism/lib/python3.6/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [83]:
#smallest range needs to be dealt with
bigger.replace({'ALL_RATE_1516': '6-9'}, '6', inplace=True)

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [95]:
bigger['ALL_RATE_1516'][0].split('-')

['92']

In [104]:
bigger['grad_rate_bin'].unique()

array(['92', '95', '85', '80', '94', '97', '96', '90', '75', '83', '70',
       '89', '87', '91', '84', '79', '77', '69', '68', '81', '93', '88',
       '98', '82', '86', '76', '78', '73', '66', '74', '60', '11', '50',
       '35', '10', '30', '40', '65', '45', '20', '25', '33', '24', '5',
       '14', '6', '55', '19', '67', '1', '71', '15', '99', '22', '46',
       '18', '49', '52', '51', '62', '53', '37', '56', '29', '38', '72',
       '63', '61', '47', '48', '64', '21', '9', '13', '58', '8', '39',
       '54', '57', '59', '42', '4', '28', '36', '3', '17', '34', '12',
       '23', '27', '32', '41'], dtype=object)

In [106]:
bigger

,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,LEA_STATE_NAME,...,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_rate_bin,grad_slice
0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,ALABAMA,...,78.0,0.060232,0.0,0.000000,2811.937359,0.000000,0.003475,0.378788,92,92
1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,95,AL,ALABAMA,...,10.0,0.018553,2.0,0.003711,4825.189777,0.000000,0.002783,0.100000,95,95
2,2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,AL,ALABAMA,...,18.0,0.030303,5.0,0.008418,5317.932795,0.000000,0.001684,0.105263,85,85
3,3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,AL,ALABAMA,...,10.0,0.021786,0.0,0.000000,5909.375686,0.000000,0.002179,0.068966,80,80
4,4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,AL,ALABAMA,...,8.0,0.022039,0.0,0.000000,3962.305785,0.000000,0.002755,0.101010,85,85
5,5,5,ALABAMA,Hoover City,10000700251,Hoover High Sch,714,92,AL,ALABAMA,...,160.0,0.053927,1.0,0.000337,434.833734,0.019066,0.001685,0.333651,92,92
7,7,7,ALABAMA,Hoover City,10000701456,Spain Park High Sch,412,94,AL,ALABAMA,...,232.0,0.138342,0.0,0.000000,511.925343,0.016920,0.001789,0.346870,94,94
8,8,8,ALABAMA,Madison City,10000800831,Bob Jones High Sch,451,97,AL,ALABAMA,...,72.0,0.041002,2.0,0.001139,72676.817158,0.000000,0.002278,0.019608,97,97
9,9,9,ALABAMA,Madison City,10000802198,James Clemens High School,404,96,AL,ALABAMA,...,134.0,0.078454,0.0,0.000000,3478.029292,0.000000,0.002635,0.232804,96,96
10,10,10,ALABAMA,Leeds City,10001102096,Leeds High Sch,123,90-94,AL,ALABAMA,...,56.0,0.119403,0.0,0.000000,3966.648380,0.000000,0.002132,0.343750,90,90


In [107]:
bigger.reset_index(inplace=True)

In [121]:
# run it with current_row = int(bigger.loc[idx, 'grad_rate_bin'])

for idx in range(len(bigger['grad_slice'])):
#     print(bigger.loc[idx, 'grad_slice'])
    current_row = int(bigger.loc[idx, 'grad_rate_bin'])
#     bigger.loc[idx, 'grad_slice'] = bigger.loc[idx, 'grad_rate_bin']
    if (current_row >= 90):
         bigger.loc[idx, 'grad_slice'] = '90%+'
    elif ((current_row < 90) and  (current_row >= 80)):
        bigger.loc[idx, 'grad_slice'] = '80-89%'
    elif ((current_row < 80) and  (current_row >= 70)):
        bigger.loc[idx, 'grad_slice'] = '70-79%'
    elif ((current_row < 70) and  (current_row >= 60)):
        bigger.loc[idx, 'grad_slice'] = '60-69%'
    elif ((current_row < 60) and  (current_row >= 50)):
        bigger.loc[idx, 'grad_slice'] = '50-59%'
    elif ((current_row < 50) and  (current_row >= 40)):
        bigger.loc[idx, 'grad_slice'] = '40-49%'
    elif ((current_row < 40) and  (current_row >= 30)):
        bigger.loc[idx, 'grad_slice'] = '30-39%'
    elif ((current_row < 30) and  (current_row >= 20)):
        bigger.loc[idx, 'grad_slice'] = '20-29%'
    elif ((current_row < 20) and  (current_row >= 10)):
        bigger.loc[idx, 'grad_slice'] = '10-19%'
    else:
        bigger.loc[idx, 'grad_slice'] = '0-9%'

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [123]:
#rename column headings
bigger.rename(columns={'grad_rate_bin':'grad_rate_slice', 'grad_slice':'grad_rates_bin'}, inplace=True)

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [124]:
bigger.head()

,index,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,...,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_rate_slice,grad_rates_bin
0,0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,...,78.0,0.060232,0.0,0.000000,2811.937359,0.0,0.003475,0.378788,92,90%+
1,1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,95,AL,...,10.0,0.018553,2.0,0.003711,4825.189777,0.0,0.002783,0.100000,95,90%+
2,2,2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,AL,...,18.0,0.030303,5.0,0.008418,5317.932795,0.0,0.001684,0.105263,85,80-89%
3,3,3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,AL,...,10.0,0.021786,0.0,0.000000,5909.375686,0.0,0.002179,0.068966,80,80-89%
4,4,4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,AL,...,8.0,0.022039,0.0,0.000000,3962.305785,0.0,0.002755,0.101010,85,80-89%


In [126]:
bigger['grad_rates_bin'].value_counts()

90%+      8712
80-89%    4257
70-79%    1457
60-69%     536
50-59%     352
10-19%     305
40-49%     278
20-29%     264
30-39%     264
0-9%       148
Name: grad_rates_bin, dtype: int64

In [142]:
#Juvenile Justice schools have very different graduation rates, 
#31 of 41 schools have graduation rates of less than 50%.

byRate = bigger[bigger['JJ'] == 'Yes'].groupby('grad_rates_bin')

In [143]:
byRate.count()

,index,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,...,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_rate_slice
grad_rates_bin,,,,,,,,,,,,,,,,,,,,,
0-9%,8,8,8,8,8,8,8,8,8,8,...,0,8,8,8,8,8,8,8,8,8
10-19%,15,15,15,15,15,15,15,15,15,15,...,0,15,15,15,15,14,15,15,15,15
20-29%,7,7,7,7,7,7,7,7,7,7,...,0,7,7,7,7,7,7,7,7,7
30-39%,5,5,5,5,5,5,5,5,5,5,...,0,5,5,5,5,5,5,5,5,5
50-59%,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,3,3,3,3,3
60-69%,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
70-79%,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
80-89%,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1


In [151]:
#Alternative Schools have a different graduation rate pattern. Out of 
#1,055 Alternative Schools, 629 graduate less than half of their cohort.
byRate_alt = bigger[bigger['SCH_STATUS_ALT'] == 'Yes'].groupby('grad_rates_bin')
byRate_alt.count()

,index,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,...,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_rate_slice
grad_rates_bin,,,,,,,,,,,,,,,,,,,,,
0-9%,85,85,85,85,85,85,85,85,85,85,...,2,85,85,84,84,48,61,85,61,85
10-19%,162,162,162,162,162,162,162,162,162,162,...,7,162,162,160,160,125,146,162,146,162
20-29%,128,128,128,128,128,128,128,128,128,128,...,6,128,128,127,127,119,125,128,125,128
30-39%,128,128,128,128,128,128,128,128,128,128,...,10,128,128,127,127,123,126,128,126,128
40-49%,126,126,126,126,126,126,126,126,126,126,...,5,126,126,123,123,124,126,126,126,126
50-59%,106,106,106,106,106,106,106,106,106,106,...,7,106,106,106,106,106,105,106,105,106
60-69%,97,97,97,97,97,97,97,97,97,97,...,2,97,97,95,95,95,96,97,96,97
70-79%,96,96,96,96,96,96,96,96,96,96,...,3,96,96,95,95,93,94,96,94,96
80-89%,79,79,79,79,79,79,79,79,79,79,...,4,79,79,79,79,79,79,79,79,79


In [152]:
#The 75 Special Education Schools have a different graduation rate pattern. Out of 
#30 Special Education Schools graduate less than half of their cohort; however,
#25 Special Education Schools graduate 90% or more of their cohort.
byRate_sped = bigger[bigger['SCH_STATUS_SPED'] == 'Yes'].groupby('grad_rates_bin')
byRate_sped.count()

,index,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,...,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_rate_slice
grad_rates_bin,,,,,,,,,,,,,,,,,,,,,
0-9%,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,3,3,3,3,3
10-19%,16,16,16,16,16,16,16,16,16,16,...,1,16,16,16,16,15,15,16,15,16
20-29%,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,3,3,3,3,3
30-39%,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,2,3,3,3,3
40-49%,5,5,5,5,5,5,5,5,5,5,...,2,5,5,5,5,5,5,5,5,5
50-59%,2,2,2,2,2,2,2,2,2,2,...,0,2,2,2,2,2,2,2,2,2
60-69%,4,4,4,4,4,4,4,4,4,4,...,2,4,4,4,4,3,4,4,4,4
70-79%,5,5,5,5,5,5,5,5,5,5,...,4,5,5,5,5,5,5,5,5,5
80-89%,9,9,9,9,9,9,9,9,9,9,...,7,9,9,9,9,9,9,9,9,9


In [87]:
#Just take the first part of the range
bigger['grad_slice'] = bigger['ALL_RATE_1516'].str[:2]

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [91]:
#convert string number to integers
type(bigger['grad_slice'])

pandas.core.series.Series

In [89]:
bigger['grad_rate_bin'] = pd.cut(bigger['grad_slice'], [0, 9, 19, 29, 39, 49, 59, 69, 79, 89, 100], labels=['0-9%', '10-19%', '20-29%', '30-39%', '40-49%', '50-59%', '60-69%', '70-79%', '80-89%', '90%+'])

TypeError: '<' not supported between instances of 'int' and 'str'

In [ ]:
df['bin'] = pd.cut(df['1'], [0, 50, 100,200], labels=['0-50', '50-100', '100-200'])


In [62]:
bigger['ALL_RATE_1516'].unique()

array(['92', '95', '85-89', '80-84', '94', '97', '96', '90-94', '75-79',
       '90', '83', '70-79', '89', '80-89', '87', '75', '91', '84', '80',
       '79', '77', '69', '68', '70-74', '81', '93', '88', '98', '82',
       '86', '85', '76', '78', '73', '66', '74', '60-64', '11-19',
       '50-59', '35-39', '10', '30-39', '60-69', '40-49', '65-69',
       '45-49', '30-34', '20-24', '40-44', '20-29', '25-29', '33', '24',
       '50-54', '5', '14', '6', '55-59', '10-14', '19', '67', '1', '71',
       '15-19', '99', '22', '46', '25', '15', '18', '49', '50', '52',
       '51', '62', '53', '37', '11', '56', '30', '29', '40', '38', '72',
       '63', '61', '47', '45', '48', '64', '35', '21', '9', '13', '58',
       '8', '39', '55', '54', '57', '65', '59', '42', '20', '4', '28',
       '36', '3', '70', '17', '34', '60', '12', '23', '27', '32', '41'],
      dtype=object)

In [ ]:
# CatWin.Game<-cut(dataset1$AdjWin.Game, breaks=c(0,0.4,0.5,0.6,0.7,0.8,1),label=c("<40%","41-50%","51-60%",">61-70%","71-80%",">80%"), right=FALSE)
# CatWin.Game [0:100]
# CatWin.Game2.<-relevel(CatWin.Game, "41-50%")